In [1]:
import warnings
from pathlib import Path
from joblib import delayed, Parallel

import librosa
import audioread
import soundfile as sf

import pandas as pd

import IPython
from scipy.io import wavfile
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
TRAIN_AUDIO_DIR = Path("../../../input/birdsong-recognition/train_audio_resampled/")
TRAIN_DENOISED_DIR = Path("../../../input/birdsong-recognition/train_audio_resampled_denoised/")

NUM_THREAD = 8  # for joblib.Parallel

# # read train.csv
train = pd.read_csv("../../../input/birdsong-recognition/train_audio_resampled/train_mod.csv")

# # extract "ebird_code" and  "filename"
train_audio_infos = train[["ebird_code", "resampled_filename"]].values.tolist()

# # make directories for saving denoised audio
TRAIN_DENOISED_DIR.mkdir(parents=True)
for ebird_code in train.ebird_code.unique():
    ebird_dir = TRAIN_DENOISED_DIR / ebird_code
    ebird_dir.mkdir()

In [5]:
# define denoiser

def _stft(y, n_fft, hop_length, win_length):
    return librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length, win_length=win_length)


def _istft(y, hop_length, win_length):
    return librosa.istft(y, hop_length, win_length)


def _amp_to_db(x):
    return librosa.core.amplitude_to_db(x, ref=1.0, amin=1e-20, top_db=80.0)


def _db_to_amp(x,):
    return librosa.core.db_to_amplitude(x, ref=1.0)

def removeNoise(
    audio_clip,
    noise_clip,
    n_grad_freq=2,
    n_grad_time=4,
    n_fft=2048,
    win_length=2048,
    hop_length=512,
    n_std_thresh=1.5,
    prop_decrease=1.0,
):
    """Remove noise from audio based upon a clip containing only noise

    Args:
        audio_clip (array): The first parameter.
        noise_clip (array): The second parameter.
        n_grad_freq (int): how many frequency channels to smooth over with the mask.
        n_grad_time (int): how many time channels to smooth over with the mask.
        n_fft (int): number audio of frames between STFT columns.
        win_length (int): Each frame of audio is windowed by `window()`. The window will be of length `win_length` and then padded with zeros to match `n_fft`..
        hop_length (int):number audio of frames between STFT columns.
        n_std_thresh (int): how many standard deviations louder than the mean dB of the noise (at each frequency level) to be considered signal
        prop_decrease (float): To what extent should you decrease noise (1 = all, 0 = none)

    Returns:
        array: The recovered signal with noise subtracted

    """
    # STFT over noise
    noise_stft = _stft(noise_clip, n_fft, hop_length, win_length)
    noise_stft_db = _amp_to_db(np.abs(noise_stft))  # convert to dB
    # Calculate statistics over noise
    mean_freq_noise = np.mean(noise_stft_db, axis=1)
    std_freq_noise = np.std(noise_stft_db, axis=1)
    noise_thresh = mean_freq_noise + std_freq_noise * n_std_thresh
    
    sig_stft = _stft(audio_clip, n_fft, hop_length, win_length)
    sig_stft_db = _amp_to_db(np.abs(sig_stft))
    # Calculate value to mask dB to
    mask_gain_dB = np.min(_amp_to_db(np.abs(sig_stft)))
    # Create a smoothing filter for the mask in time and frequency
    smoothing_filter = np.outer(
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_freq + 1, endpoint=False),
                np.linspace(1, 0, n_grad_freq + 2),
            ]
        )[1:-1],
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_time + 1, endpoint=False),
                np.linspace(1, 0, n_grad_time + 2),
            ]
        )[1:-1],
    )
    smoothing_filter = smoothing_filter / np.sum(smoothing_filter)
    # calculate the threshold for each frequency/time bin
    db_thresh = np.repeat(
        np.reshape(noise_thresh, [1, len(mean_freq_noise)]),
        np.shape(sig_stft_db)[1],
        axis=0,
    ).T
    # mask if the signal is above the threshold
    sig_mask = sig_stft_db < db_thresh

    # convolve the mask with a smoothing filter
    sig_mask = scipy.signal.fftconvolve(sig_mask, smoothing_filter, mode="same")
    sig_mask = sig_mask * prop_decrease

    # mask the signal
    sig_stft_db_masked = (
        sig_stft_db * (1 - sig_mask)
        + np.ones(np.shape(mask_gain_dB)) * mask_gain_dB * sig_mask
    )  # mask real
    sig_imag_masked = np.imag(sig_stft) * (1 - sig_mask)
    sig_stft_amp = (_db_to_amp(sig_stft_db_masked) * np.sign(sig_stft)) + (
        1j * sig_imag_masked
    )

    # recover the signal
    recovered_signal = _istft(sig_stft_amp, hop_length, win_length)
    recovered_spec = _amp_to_db(
        np.abs(_stft(recovered_signal, n_fft, hop_length, win_length))
    )

    return recovered_signal

In [159]:
# # define noise clip selector

def brute(data, sr):
    mn = INF
    ret = -1
    sm = 0
    for i in range(sr):
        sm += abs(data[i])

    for start in range(min(len(data)-sr, sr*10)):
        end = start + sr
        if(start > 0):
            sm += abs(data[end-1])
            sm -= abs(data[start-1])
        if(sm < mn):
            mn = sm
            ret = start

    return ret

def select_noiseclip(data, sr):
    mn_mean = 1001001001
    ret = -1
    data = data.tolist()
    sm = 0
    mx = -1
    mx_idx = -1
    for i in range(sr):
        sm += abs(data[i])
        if(data[i] > mx):
            mx = data[i]
            mx_idx = i

    range_mx = -1
    range_mx_idx = -1
    for i in range(mx_idx+1, sr):
        if(data[i] > range_mx):
            range_mx = data[i]
            range_mx_idx = i

    for start in range(min(len(data)-32000, sr*10)):
        end = start + sr

        if(start > 0):
            sm += abs(data[end-1])
            sm -= abs(data[start-1])
            if(data[end-1] > mx):
                range_mx = mx
                range_mx_idx = mx_idx
                mx = data[end-1]
                mx_idx = end-1
            elif(data[end-1] > range_mx):
                range_mx = data[end-1]
                range_mx_idx = end-1

        dmean = sm / sr

        if(start > mx_idx):
            mx = range_mx
            mx_idx = range_mx_idx

        dmx = mx

        if(dmean < mn_mean):
            if(7*dmean > dmx): 
                mn_mean = dmean
                ret = start

    if(ret < 0):
        ret = brute(data, sr)
    
    return ret, ret + sr

In [166]:
# # define denoise function
warnings.simplefilter("ignore")
def denoise(ebird_code: str, filename: str):    
    audio_dir = TRAIN_AUDIO_DIR
    denoised_dir = TRAIN_DENOISED_DIR
    ebird_dir = denoised_dir / ebird_code

    try:
        data, sr = sf.read(
            audio_dir / ebird_code / filename
        )
        noise_start, noise_end = select_noiseclip(data, sr)
        denoised = removeNoise(data, data[noise_start: noise_end])
        
        sf.write(ebird_dir / filename, denoised, samplerate=sr)
        return "OK"
    except Exception as e:
        with open(denoised_dir / "skipped.txt", "a") as f:
            file_path = str(audio_dir / ebird_code / filename)
            f.write(file_path + "\n")
        return str(e)

In [ ]:
# # denoise and save audio using Parallel
msg_list = Parallel(n_jobs=NUM_THREAD, verbose=1)(
    delayed(denoise)(ebird_code, file_name) for ebird_code, file_name in train_audio_infos)


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    6.4s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   37.6s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:  2.7min
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:  4.1min
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:  5.9min
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:  8.3min
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed: 11.0min
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed: 14.4min
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed: 17.7min
[Parallel(n_jobs=8)]: Done 6034 tasks      | elapsed: 21.3min
[Parallel(n_jobs=8)]: Done 7184 tasks      | elapsed: 24.8min
[Parallel(n_jobs=8)]: Done 8434 tasks      | elapsed: 29.3min
[Parallel(n_jobs=8)]: Done 9784 tasks      | elapsed: 34.0min
